In [1]:
import numpy as np 
import urdf2casadi.urdfparser as u2c
from urdf2casadi.geometry import plucker
from urdf_parser_py.urdf import URDF, Pose
import PyKDL as kdl
import kdl_parser_py.urdf as kdlurdf
from timeit import Timer, timeit, repeat
import rbdl
import pybullet as pb


In [2]:
sim = pb.connect(pb.DIRECT)
pb.setGravity(0,0,-9.81)
#path_to_urdf = "/home/lillmaria/urdf2casadi/examples/urdf/pantilt.urdf"
path_to_urdf = "/home/lillmaria/urdf2casadi/examples/timing/urdf4timing/2dof.urdf"

#tip = "tilt_link"
root = 'base_link'
tip = 'link2'

ok, kdl_tree = kdlurdf.treeFromFile(path_to_urdf)
kdlmodel = kdl_tree.getChain(root,tip)
    
#rbdlmodel = rbdl.loadModel(path_to_urdf)
pbmodel = pb.loadURDF(path_to_urdf,globalScaling=1, basePosition = (0, 0, 10), baseOrientation = (0, 0, 0, 10), useFixedBase=True, flags=pb.URDF_USE_IMPLICIT_CYLINDER|pb.URDF_USE_INERTIA_FROM_FILE|pb.URDF_INITIALIZE_SAT_FEATURES )
asd = u2c.URDFparser()
asd.from_file(path_to_urdf)
    
jlist, names, qmax, qmin = asd.get_joint_info(root, tip)
njoints = len(jlist)
    
gravity = [0, 0, -9.81]
    
#rbdl declarations
q_np = np.zeros(njoints)
qdot_np = np.zeros(njoints)
qddot_np = np.zeros(njoints)
id_rbdl = np.zeros(njoints)
    
#u2c and pybullet declarations
q = [None]*njoints
qdot = [None]*njoints
qddot = [None]*njoints
id_pb = [None]*njoints
c_pb = [None]*njoints

gravity_kdl = kdl.Vector()
gravity_kdl[2] = -9.81
q_kdl = kdl.JntArray(njoints)
qdot_kdl = kdl.JntArray(njoints)
g_kdl = kdl.JntArray(njoints)
c_kdl = kdl.JntArray(njoints)
    
id_sym = asd.get_inverse_dynamics_rnea(root, tip, gravity)
c_sym = asd.get_coriolis_rnea(root, tip)

The root link base_link has an inertia specified in the URDF, but KDL does not support a root link with an inertia.  As a workaround, you can add an extra dummy link to your URDF.
/home/lillmaria/urdf2casadi/examples/timing/urdf4timing/2dof.urdf


In [3]:
for i in range(njoints):
    q[i] = (qmax[i] - qmin[i])*np.random.rand()-(qmax[i] - qmin[i])/2
    q_kdl[i] = q[i]
    qdot[i] = 0#(qmax[i] - qmin[i])*np.random.rand()-(qmax[i] - qmin[i])/2
    qddot[i] = 0#(qmax[i] - qmin[i])*np.random.rand()-(qmax[i] - qmin[i])/2

    
    
id_u2c = id_sym(q, qdot, qddot)
print "g u2c:", id_u2c

id_pb = pb.calculateInverseDynamics(pbmodel, q, qdot, qddot)
print "g pb:", id_pb

kdl.ChainDynParam(kdlmodel, gravity_kdl).JntToGravity(q_kdl, g_kdl)
print "g kdl", g_kdl

g u2c: [2.40763, 11.0924]
g pb: (2.407627808328206, 11.09235825826987)
g kdl 2.40763
11.0924


In [4]:
for i in range(njoints):
    q[i] = (qmax[i] - qmin[i])*np.random.rand()-(qmax[i] - qmin[i])/2
    q_kdl[i] = q[i]
    qdot[i] = (qmax[i] - qmin[i])*np.random.rand()-(qmax[i] - qmin[i])/2
    qdot_kdl[i] = qdot[i]
    qddot[i] = (qmax[i] - qmin[i])*np.random.rand()-(qmax[i] - qmin[i])/2

c_u2c = id_sym(q, qdot, qddot)
print "c u2c:", c_u2c

c_pb = pb.calculateInverseDynamics(pbmodel, q, qdot, qddot)
print "c pb:", c_pb

kdl.ChainDynParam(kdlmodel, gravity_kdl).JntToCoriolis(q_kdl, qdot_kdl, c_kdl)
print "c kdl", c_kdl

c u2c: [9.98406, 2.65551]
c pb: (9.984062385537788, 2.655511809968429)
c kdl -6.46365
 0.33888


In [5]:
import casadi as cs

def u2c2np(asd):
    return cs.Function("temp",[],[asd])()["o0"].toarray()

def kdl2np(asd):
    x = []
    for i in range(njoints):
        x.append(asd[i])
    return np.asarray(x)

error = np.zeros(njoints)

n_itr = 1000

for i in range(n_itr):
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        q_kdl[j] = q[j]
        qdot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_kdl[j] = qdot[j]
        qddot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2


   
    c_u2c = id_sym(q, qdot, qddot)
    #print "c u2c:", c_u2c

    c_pb = pb.calculateInverseDynamics(pbmodel, q, qdot, qddot)
    #print "c pb:", c_pb


    for tau_idx in range(njoints):
        error[tau_idx] += np.absolute((kdl2np(c_pb)[tau_idx] - u2c2np(c_u2c)[tau_idx]))

print "Errors in coriolis forces with",n_itr, "iterations and comparing against KDL:\n", error

sum_error = 0
for err in range(njoints):
    sum_error += error[err]
print "Sum of errors:\n", sum_error


Errors in coriolis forces with 1000 iterations and comparing against KDL:
[2.41495712e-12 1.47754031e-12]
Sum of errors:
3.89249743548703e-12
